# 2.8- Sequencing data and time series analysis

### Learning goals for today
1. Load, visualize and understand RNA sequencing data
2. Annotating scRNAseq data using marker genes
3. Bonus: Analyzing time series data

---
### How to use this notebook during class
- Follow along as we go
- Use your **Cards** to indicate where you're at:
    - A **🟩Green card** means you are caught up with Max and **ready to help your classmates**
    - A **🟥Red card** means you are stuck and need help
- <span style='color:red;'>EXERCISE</span> — work on this problem by yourself, or try with a partner if you get stuck
---


First let's import some libraries we will use today. Remember if you don't have a library installed you can pip install it with:

%pip install *package_name*

## 1) scRNAseq data - looking at it

Download `tags.csv` and `ct.csv` from brightspace or the course github

Data originally from: https://www.pnas.org/doi/full/10.1073/pnas.1507125112



In [ ]:
%pip install scanpy
%pip install igraph

Generally it is best to not keep data in your code repository, and GitHub cannot sync large files. So let's point our code to the file path on the computer

First we'll use pandas to load the data. There are two things we'll load:
1. The gene counts matrix, which has cells along the rows and genes along the columns
2. The cell type array, which has labels for each annotated cell type 

These data have already been annotated. Let's look at the cell types in the annotation

This is a big matrix, and scanpy uses a special data type to hold this and run calcuations on it called an Annotated Data (AnnData) object

Let's make an AnnData (Annotated Data) object from our two dataframes

Let's do some standard preprocessing steps to remove cells that don't have many genes, and genes that don't show up in many cells

Notice that when we do stuff to adata, it modifies the object **in place** and adds new observations to the table

The next step is to normalize the read counts within a cell to that cell's median reads. That lets us compare amount of gene expression across cells that maybe just have more mRNA around for whatever reason.

Then we'll transform to log(count+1). Log transforming helps make the variances more comparable for genes that may differ in abundance over orders of magnitude. 

**Feature selection:** Let's add tags to indicate "highly variable genes," to help us focus on genes that are differentially expressed across the cells. We will use this for visualizing lower dimensional representations (like UMAP) and for clustering

Let's do PCA to get a sense of the dimensionality of the data

Let's look at a lower dimensional representation of the gene expression profiles of these cells using UMAP. We will use pre-annotated cell types to color the points

## 2) scRNAseq data - clustering and annotating
What if we didn't have cell type annotations? Let's see how we could make our own using clustering and known marker genes


Now let's cluster our cells using the Leiden method. We can control how fine-scale the clustering is with the `resolution` parameter.

### <span style='color:red;'>EXERCISE 1</span>: Find a good resolution value  (5 min)

Play with the `resolution` parameter to find a good value that gives use approximately the right number of clusters. You can iterate on this while trying to assign clusters to cell types


Suppose we have marker genes for each of the cell types we are interested in. We can tell scanpy about these marker genes using a dictionary like this one:

In [ ]:
marker_genes_dict = {
    "OPC": ["PCDH15", "MEGF11", "LHFPL3", "PDGFRA"],
    "Astrocytes": ["SLC14A1", "GLIS3", "GLI3", "CTH", "HGF"],
    "Endothelial": ["APOLD1", "FLT1", "A2M", "TM4SF1"],
    "microglia": ["CCL4", "GPR183", "CD83", "CD14", "CSF1R"],
    "neurons": ["KCNK1", "NELL1", "COBL", "DCN", "DLX2", "DLX5", "SST", "PNOC"],
    "oligodendrocytes": ["DAAM2", "ASPA", "MAL", "MAG", "GSN"],
}

Now let's see how our clusters line up to real cell types using a dotplot

In [ ]:
cluster_mapping_dictionary =     {
                                  "0": "?",
                                  "1": "Endothelial",
                                  "2": "Astrocytes",
                                  "3": "Neurons",
                                  "4": "Oligodendrocytes",
                                  "5": "OPC",
                                 }

Let's add an observation to the andata object to hold our putative cell types. We'll do this using the pandas .map() function on our dictionary

Then we'll look at the UMAP, colored by putative cell type

You can imagine that for your own data you would want to iterate on clustering, labeling, and identifying marker genes until you are happy with your clusters

Let's take a rough look at differentially expressed genes among these groups

Let's make volcano plot

# BONUS: time series analysis

## 3) Time series data -  Examining extracellular spiking data

### Detect spike times
Let's say we want to detect the times when the neuron fired an action potential (spike). 

Steps:
1. Filter the signal: remove low-frequency changes in baseline using a **high-pass filter**
2. Find the points where the trace goes beyond some threshold value
3. Make sure you aren't double counting spikes!


## 4) Spike triggered average of a neuron

Reliable spike detection is beyond the scope of this class, so we're going to skip some intervening steps and start with already detected spikes.
A common way to represent spike times is a vector that is zero everywhere except the time points where the spike occurred.

Let's load some processed spike time data, alongside the visual stimulus that was shown to the neuron. These data are from a neuron in blowfly called H1 that is sensitive to motion in the visual world. The stimulus is reported as the rotational velocity (in degrees/second). Both the stimulus and spike times have a sampling period of 2 msec.

`'stim'` is the stimulus

`'rho'` is the spike binary vector

Let's make some variables to be able to refer to each of these a bit more easily. We'll also make a vector of timestamps based on the knowledge that each time step is 2 msec


Now let's jsut plot a bit of the stim and response trace to see what we're looking at

We're going to figure out what makes this neuron spike. To do that we will compute the **spike triggered average stimulus** this, as the name implies is the average stimulus that triggers a spike. The strategy is straightforward:
1. Define some window of time to look backwards (let's say 300 datapoints, or 600 msec)
2. Find all the spike times
3. Pull out the stimulus for the 300 datapoints preceding each spike
4. Average all the stimuli together

Now let's plot our spike triggered average. To visualize it, we need a time vector for the x axis...